In [117]:
import os
import numpy as np
import pandas as pd
from custom import utils
from collections import Counter
import inspect

### Headers for subsetting brain data

In [113]:
def most_handed(data):
    counts = Counter(data)
    most = max(counts.items())[0]
    data = data.copy().fillna(0)
    data[data == 0] = 'Right'
    both = ['Both','Mixed','Either','Ambidextrous']
    for hand in both:
        data[data == hand] = 'Left'
    return data

def preprocess(brain_path, snp_path, bcvar, brain_cols):
    # load data
    brain_data = pd.read_hdf(brain_path)
    snp_data = pd.read_hdf(snp_path)
    # get the group status
    gr = brain_data.GROUP.values
    cnt_scz = np.logical_or(gr == 'Control', gr == 'Schizophrenia')
    # subset by indexes cnt_scz
    brain_data = brain_data.iloc[cnt_scz, :]
    snp_data = snp_data.iloc[cnt_scz, :]
    # create set of covariates
    icv = 'EstimatedTotalIntraCranialVol'
    cov_set1 = pd.DataFrame(
        data=np.hstack((snp_data.SEX.values[:, None],
                        brain_data.AGE_MRI.values[:, None],
                        brain_data[icv].values[:, None])),
        columns=['SEX','AGE','EstimatedTotalIntraCranialVol'])
    cov_set1 = cov_set1.fillna(0)
    cov_set1[cov_set1.AGE == 0] = cov_set1.AGE.mean()
    cov_site = utils.make_non_singular(utils.encoder(brain_data.STUDY.values))
    cov_site_cols = ['site{}'.format(i) for i in range(cov_site.shape[1])]
    cov_site = pd.DataFrame(data=cov_site, columns=cov_site_cols)
    cov_hand = utils.encoder(most_handed(brain_data.HANDED))[:, -1][:, None]
    cov_hand_cols = ['handed{}'.format(i) for i in range(cov_hand.shape[1])]
    cov_hand = pd.DataFrame(data=cov_hand, columns=cov_hand_cols)
    cvars = pd.concat([cov_set1, cov_site, cov_hand], axis=1)
    y = np.array([0 if i == 'Control' else 1 for i in brain_data.GROUP.values])
    return {'Z': cvars.values, 
            'I': brain_data[brain_cols].values, 
            'G': snp_data.iloc[:, 1:-5].values,
            'colnames': snp_data.iloc[:, 1:-5].columns.values,
            'y':y}, {'Z_cols':cvars.columns.values,
                     'I_cols':brain_cols,
                     'G_cols': snp_data.iloc[:, 1:-5].columns.values}

def save_preprocessed(preproc_data_dict, preproc_data_dict_col, save_path, dn, cn):
    save_dict = os.path.join(save_path, dn)
    save_cols = os.path.join(save_path, cn)
    scipy.io.savemat(save_dict, mdict=preproc_data_dict)
    utils.save_pickle(preproc_data_dict_col, save_cols)
    return save_dict, save_cols

def cv_maker(data_path, save_path):
    import scipy.io
    from sklearn.model_selection import StratifiedKFold
    X = scipy.io.loadmat(data_path)['I']
    y = scipy.io.loadmat(data_path)['y'][0]
    cv = StratifiedKFold(n_splits=10, random_state=1)
    train_idx, test_idx = {}, {}
    for idx, (train, test) in enumerate(cv.split(X, y)):
        train_idx['train_{}'.format(idx + 1)] = train + 1
        test_idx['test_{}'.format(idx + 1)] = test + 1
    scipy.io.savemat(save_path, mdict={"train":train_idx, "test":test_idx})
    return save_path



In [121]:
"""
headers_dir = "/storage/gablab001/data/genus/fs_cog/pred_diag/column_headers"
brain_cols = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
brain_path = "brain_N1547_P5927_matched.hdf5py"
snp_path = "genomic_N1547_P100006_matched.hdf5py"
bcv = ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']
all_data, all_cols = preprocess(brain_path, snp_path, bcv, brain_cols)
""";

In [122]:
from nipype import Function, Node, Workflow, IdentityInterface

In [ ]:
CV_maker = Node(interface=Function(
    input_names = ['data_path', 'save_path'],
    output_names = ['save_path'],
    function = cv_maker
), name = 'CV_maker')

#CV_maker.inputs.data_path = "/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat"
#CV_maker.inputs.save_path = "/storage/gablab001/data/genus/brain_genomic_bayes/cv_idxs.mat"

#cv_maker("/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat",
#         "/storage/gablab001/data/genus/brain_genomic_bayes/cv_idxs.mat")

wf = Workflow(name='brain_bcv')
wf.base_dir = "/om/user/ysa"

Iternode = Node(IdentityInterface(fields=['col_idx', 'cv_idx']), name = 'Iternode')
Iternode.iterables = [('col_idx', np.arange(170) + 1), ('cv_idx', np.arange(10) + 1)]

def run_bayes(in_file, cv_file, cv_idx, col_idx, out_file):
    import cPickle as pickle
    import numpy as np
    import os
    import nipype.interfaces.matlab as Matlab
    def outnames(col, out):
        return os.path.join(out, '{}.mat'.format(col))
    col_names = np.genfromtxt("/storage/gablab001/data/genus/brain_genomic_bayes/170_columns.txt", dtype=str)
    col_save_name = col_names[col_idx - 1] + "_{}_{}_BF".format(cv_idx, col_idx)
    with open("/storage/gablab001/data/genus/brain_genomic_bayes/bayes_reg.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_file = outnames(in_file[:-4] + col_save_name, out_file)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.paths = [
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs/varbvs-MATLAB',
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs',
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs/varbvs-R']
    matlab.inputs.script = script.format(in_file, cv_file, cv_idx, col_idx, mat_file)
    res = matlab.run()
    return mat_file

Run_bayes = Node(interface=Function(
    input_names = ['in_file', 'cv_file','cv_idx',
                   'col_idx','out_file'],
    output_names = ['mat_file'],
    function = run_bayes
), name='Run_bayes')

Run_bayes.inputs.in_file = "/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat"
Run_bayes.inputs.cv_file = "/storage/gablab001/data/genus/brain_genomic_bayes/cv_idxs.mat"
Run_bayes.inputs.out_file = "/storage/gablab001/data/genus/brain_genomic_bayes/"

wf.connect(Iternode, 'cv_idx', Run_bayes, 'cv_idx')
wf.connect(Iternode, 'col_idx', Run_bayes, 'col_idx')

wf.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=2G -t 05:00:00', 'max_jobs': 50})